In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from define_CNN_class import MyResNetModel
import tensorflow as tf
from keras.layers import (Dense,Conv1D,BatchNormalization,GlobalAveragePooling1D
                          ,MaxPooling1D,AveragePooling1D,Multiply,Add)



In [ ]:
timeline = np.arange(310)*0.00064


def normalization(data):
    nomalized =(data-np.min(data))/(np.max(data)-np.min(data))
    return nomalized

def make_datas(timeline):#モデルの学習用データを作る関数
 
    #T2を決定
    T2_0=np.random.uniform(0.0200,0.035)
    T2_1=np.random.uniform(0.031,0.07)
    T2_2=np.random.uniform(0.07,0.145)
    T2_3=np.random.uniform(0.145,0.195)#ここは3成分に修正

    #ワイブル係数を決定
    W_0=1 + np.random.uniform(0,1)
    #存在比を決定
    a=np.random.uniform(0,100)
    b=np.random.uniform(0,0.5)
    c=np.random.uniform(0,1)
    d=np.random.uniform(0,1)


    abcd_sum=a+b+c+d

    a=a/abcd_sum
    b=b/abcd_sum
    c=c/abcd_sum
    d=d/abcd_sum


    #print(f"a:{a},b:{b},c:{c}")
    #print(f"T2_0:{T2_0},T2_1:{T2_1},T2_2:{T2_2}")
    #print(f"W_0{W_0}")
    raw_data = a*np.exp(-W_0*(timeline/T2_0)**W_0)+b*np.exp(np.array(timeline)/(-T2_1))+c*np.exp(timeline/(-T2_2))+d*np.exp(timeline*0.01/(-T2_3))
    params=np.array([a,b,c,d,T2_0,T2_1,T2_2,T2_3,W_0])


    #正規分布で出力されるノイズ項に対して異なる定数をかけて積算の異なるデータの再現をしている
    noise_rate=0.2
    ramdom_noise_rate=np.random.uniform(0,0.05)

    #生成した曲線にノイズ項を付与する
    noisd_data = raw_data + (np.random.randn(len(timeline))*noise_rate*ramdom_noise_rate)

    con_array=np.stack((raw_data,noisd_data))
    con_array = normalization(con_array)

    raw_data = con_array[0,: ]
    noisd_data = con_array[1,:]

    return raw_data,noisd_data


raw_data,noisd_data,=make_datas(timeline)
plt.plot(timeline, noisd_data,label="hard")
plt.plot(timeline, raw_data,label="raw",color="black")

#生成した曲線の様子を600点目まで確認。横軸時間、縦軸データで表示

plt.xlabel("time(ms)")
plt.ylabel("Relaxation Rate")
plt.legend()
plt.show()


In [ ]:
output_data =[] #ノイズを付与する前のデータが格納されている配列。正解データに相当
input_data = []#CNNの問題データを保持する配列。ここではノイズの強いデータが代入されている


data_size=30000
for n in range(data_size):
    raw,noised_data=make_datas(timeline)
    input_data.append(noised_data)
    output_data.append(raw)


#ここまででlist型オブジェクトにtrainまたはtestのnp.arrayが格納されている

input_data = np.array(input_data)#listにnp.arrayを格納している状態を二次元np.arrayに変更する
output_data = np.array(output_data)

print(input_data.shape)
print(output_data.shape)

plt.plot(timeline,input_data[0,:])
plt.plot(timeline,output_data[0,:])
plt.show()

train_X = np.reshape(input_data, (-1, len(timeline), 1))#強ノイズデータ
No_noise=np.reshape(output_data, (-1,len(timeline), 1))

print(train_X.shape)
print(No_noise.shape)


In [ ]:


model = MyResNetModel()
your_input_length=311
your_input_channels=1
model.build((None, your_input_length, your_input_channels))
model.compile(optimizer="adam", loss=["mse",None,None],loss_weights=[1,0,0])




In [ ]:
epochs = 30 #エポック数を定義
tf.keras.backend.clear_session()

history = model.fit(train_X[:, :, :], No_noise[:, :, :], validation_split=0.1 , epochs = epochs )

#validation_split=0.1は学習データの末尾の10％を検証用にカットする引数。交差検証を行えるわけではない
#10000点の全体のうち9000点で学習している
model.summary()

#If you save the model, you can load it in other files.

In [ ]:
plt.plot(range(epochs), history.history['loss'], label='train')
plt.plot(range(epochs), history.history['val_loss'], label='test')
plt.xlabel('epoch')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
#新しい方
start = int(data_size*0.9)
pred=[]


res1 = model.predict(train_X[start:, :, :])[0]

print(res1.shape)

In [ ]:
#新しい方
for i in range(40):
    plt.plot(figsize=(100, 60))
    plt.title(f"{i+1}")

    plt.plot(timeline, train_X[i+start,:, :].reshape(-1), label='noise')
    plt.plot(timeline, No_noise[i+start,:, :].reshape(-1), label='No_noise')
    plt.plot(timeline, res1[i, :].reshape(-1), label='predict',color="black")

    plt.legend()
    plt.show()


In [ ]:
for i in range(40):
    plt.plot(figsize=(100, 60))
    plt.title(f"{i+1}_(log)")

    plt.plot(timeline, np.log(train_X[i+start,:, :].reshape(-1)), label='noise')
    plt.plot(timeline, np.log(No_noise[i+start,:, :].reshape(-1)), label='No_noise')
    plt.plot(timeline, np.log(res1[i, :].reshape(-1)), label='predict',color="black")

    plt.legend()
    plt.show()